# [STEP2] ChatGPTによるSQL生成

データベース構造から特定のカラムだけを抽出するSQLを、ChatGPTを用いて生成できるかを検証する。

In [ ]:
# Pythonライブラリインストール
# ※Python 3.10.x　使用推奨
!python --version
!pip install python-dotenv
!pip install --upgrade openai
!pip install openai[datalib]

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install scikit-learn
!pip install sqlalchemy


## 環境変数
supabase接続用URL,APIキーと、openai api接続用のAPIキーを設定します。
自身のopenaiアカウントからapi keyを取得してください。

https://platform.openai.com/account/api-keys

supabaseの情報は管理者にお尋ねください。

下記の例では、.envファイルに変数を書き込んで、JupiterNotebookで読み込む仕様で実装しております。

※.envファイルの作成が困難、.envファイルから値を読み込めない場合、
　os.getenv("◯◯")部分に変数値を直接書き込んでいただいても動作自体には問題ありません。

In [4]:
# 環境変数
import os
from dotenv import load_dotenv
load_dotenv()

# supabase接続用変数
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")

# OPENAI API KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

print('環境変数読み込み完了')

環境変数読み込み完了


# 処理実行
## [INPUT] 出力したいカラムリスト

STEP1での出力結果を想定

In [5]:
# INPUT 出力したいカラムリスト
# STEP1で選定されたカラム（[table name].[column name]）
input_columns = """
structures."name"
structures."acronym"
structures."id"
specimens."structure-id"
experiments."id"
specimens."experiment-id"
projections."experiment-id"
projections."structure-id"
projections."projection-density"
projections."projection-volume"
"""

# データベース構造（テーブル - カラム）
database_structure_information="""
Table: experiments
- id (integer, primary key, unique)
- qc-date (text)
- red-channel (text)
- green-channel (text)
- blue-channel (text)

Table: structures
- id (integer, primary key, unique)
- name (text)
- acronym (text)
- parent-structure-id (integer)
- hemisphere-id (integer)
- st-level (integer)
- superstructures (jsonb)
- substructures (jsonb)
- neighboring-structures (jsonb)

Table: specimens
- id (integer, primary key, unique)
- experiment-id (integer, foreign key referencing experiments.id)
- donor-id (integer)
- sex (text)
- strain (text)
- age (real)
- weight (real)
- structure-id (integer, foreign key referencing structures.id)
- registration-point (text)
- coordinates-ap real null,
- coordinates-dv real null,
- coordinates-ml real null,
- angle (integer)
- injection-materials (text)
- fluor-colors (text)
- injection-method (text)
- days-post-injection (integer)

Table: projections
- id (integer, primary key, unique)
- experiment-id (integer, foreign key referencing experiments.id)
- hemisphere-id (integer)
- structure-id (integer, foreign key referencing structures.id)
- is-injection (boolean)
- normalized-projection-volume (real)
- projection-density (real),
- projection-energy (real)
- projection-intensity (real)
- projection-volume (real)
- volume (real)
"""

## OpenAI API (GPT-4)を用いてSQL生成

In [14]:
import openai
openai.api_key = openai_api_key


def generate_sql(query:str):
    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
        {"role": "system", "content": "You have a database related with Mouse Brain Connectivity that resource is Allen Brain Atlas API.\n----\n"+database_structure_information},
        {"role": "system", "content": "Return a completion consisted of only SQL"},
        {"role": "user", "content": "Generate SQL for Postgre in order to export columns below \n----\n"+query}
      ],
      temperature=0.1
    )
    return completion.choices[0].message.content

sql = generate_sql(input_columns)

print(sql)

SELECT
    structures.name,
    structures.acronym,
    structures.id AS structure_id,
    specimens.structure_id AS specimen_structure_id,
    experiments.id AS experiment_id,
    specimens.experiment_id AS specimen_experiment_id,
    projections.experiment_id AS projection_experiment_id,
    projections.structure_id AS projection_structure_id,
    projections.projection_density,
    projections.projection_volume
FROM
    structures
JOIN
    specimens ON structures.id = specimens.structure_id
JOIN
    experiments ON specimens.experiment_id = experiments.id
JOIN
    projections ON projections.experiment_id = experiments.id AND projections.structure_id = structures.id;


## SQLを適用しデータ出力

カラム名を一部修正

In [21]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
from IPython.display import display

fixed_column_names = [
  {
    "before":".structure_id",
    "after":".\"structure-id\""
  },
  {
    "before":".experiment_id",
    "after":".\"experiment-id\""
  },
  {
    "before":".projection_density",
    "after":".\"projection-density\""
  },
  {
    "before":".projection_volume",
    "after":".\"projection-volume\""
  },
  {
    "before":";",
    "after":" LIMIT 50;"
  }
]
sql_fixed = sql
for f in fixed_column_names:
  sql_fixed=sql_fixed.replace(f["before"], f["after"])

print(sql_fixed)


# Connect to the database
connection_config = {
    'user': db_user,
    'password': urllib.parse.quote_plus(db_pass),
    'host': db_host,
    'port': db_port, 
    'database': db_name
}
engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**connection_config))
df = pd.read_sql(sql=sql_fixed, con=engine)

display(df)

SELECT
    structures.name,
    structures.acronym,
    structures.id AS structure_id,
    specimens."structure-id" AS specimen_structure_id,
    experiments.id AS experiment_id,
    specimens."experiment-id" AS specimen_experiment_id,
    projections."experiment-id" AS projection_experiment_id,
    projections."structure-id" AS projection_structure_id,
    projections."projection-density",
    projections."projection-volume"
FROM
    structures
JOIN
    specimens ON structures.id = specimens."structure-id"
JOIN
    experiments ON specimens."experiment-id" = experiments.id
JOIN
    projections ON projections."experiment-id" = experiments.id AND projections."structure-id" = structures.id LIMIT 50;


,name,acronym,structure_id,specimen_structure_id,experiment_id,specimen_experiment_id,projection_experiment_id,projection_structure_id,projection-density,projection-volume
0,Dentate gyrus,DG,726,726,100141214,100141214,100141214,726,9.308600e-01,1.163210e-01
1,Dentate gyrus,DG,726,726,100141214,100141214,100141214,726,1.460160e-01,1.043780e+00
2,Primary visual area,VISp,385,385,100141219,100141219,100141219,385,9.907240e-01,2.014180e-01
3,Primary visual area,VISp,385,385,100141219,100141219,100141219,385,1.748060e-01,6.461390e-01
4,Primary visual area,VISp,385,385,100141219,100141219,100141219,385,2.978310e-03,1.161430e-02
5,Primary visual area,VISp,385,385,100141219,100141219,100141219,385,9.907240e-01,2.014180e-01
6,Primary visual area,VISp,385,385,100141219,100141219,100141219,385,8.659290e-02,6.577530e-01
7,Primary motor area,MOp,985,985,100141220,100141220,100141220,985,4.030990e-02,2.366080e-01
8,Primary motor area,MOp,985,985,100141220,100141220,100141220,985,2.260080e-01,1.243290e+00
9,Primary motor area,MOp,985,985,100141220,100141220,100141220,985,1.301490e-01,1.479890e+00
